# <center> WSI Ćwiczenie nr7 - Implementacja naiwnego klasyfikatora Bayesa</center>

### <center> Adam Wróblewski</center>


### Cel eksperymentów:
Celem eksperymentów jest stworzenie algorytmu naiwnego klasyfikatora Bayessa i zbadanie jakości klasyfikatorów dla zbioru danych Cardio Vascular Disease Detection, a także zbadanie wpływu sposobu podziału na wynik. Zastosowaną zostane 2 metody podziału:

1. standardowa - dzielimy zbiór danych na zbiór uczący, walidacyjny, testowy zgodnie z ustalonymi proporcjami.
2. k-krotna walidacja krzyżowa.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
import math

In [2]:
my_data = pd.read_csv('cardio_train.csv', sep = ';')

In [4]:
class NaiveBayes:
    def __init__(self, data, class_name):
        self.data = data
        self.X_data = None
        self.Y_data = None
        self.class_name = class_name
        self.class_values = None


        self.mean_variance_dict = {}
        self.means = []
        self.variances = []
        self.aprioris = []
        self.get_data(data)
        self.calculate_needed_variables()
    
    def get_data(self, data):
        x_data = data.drop(self.class_name, axis=1, inplace=False)
        y_data = data[self.class_name]
        self.X_data = x_data
        self.Y_data = y_data
        self.class_values = list(np.unique(y_data))
        return x_data, y_data
    
    def calculate_needed_variables(self):
        for i, class_value in enumerate(self.class_values):
            data = self.data
            X_with_c_v = (data[data[self.class_name] == class_value]).drop(self.class_name, axis=1, inplace=False)

            self.means.append(list(X_with_c_v.mean(axis=0)))
            self.variances.append(list(X_with_c_v.var(axis=0)))
            self.aprioris.append(X_with_c_v.shape[0]/data.shape[0])

    def calculate_gauss(self, x, mean, variance):
        bias = 1e-5
        return (1 / (math.sqrt(2 * math.pi) * variance + bias)) * math.exp(-((x-mean)**2 / (2 * variance**2 + bias)))
    
    def predict(self, row):
        probabilites = {}
        for i, class_val in enumerate(self.class_values):
            apriori = self.aprioris[i]
            probability = apriori
            for n, x_val in enumerate(row):
                attribute_mean = self.means[i][n]
                attribute_variance = self.variances[i][n]
                gauss = self.calculate_gauss(x_val, attribute_mean, attribute_variance)
                probability = probability * gauss
            probabilites[class_val] = probability
        
        return max(probabilites, key=probabilites.get)


    def validate(self, data):
        incorrect_predict_count = 0
        correct_predict_count = 0

        for _, row in data.iterrows():
            real_value = row[self.class_name]
            row.pop(self.class_name)
            row = list(row)
            # print(row)
            prediction = self.predict(row)
            if prediction == real_value:
                correct_predict_count += 1
            else:
                incorrect_predict_count += 1
        accuracy = correct_predict_count/(correct_predict_count+incorrect_predict_count)
        return accuracy

In [17]:
def standard_validation(data, train_factor, validate_factor):
    data = data.drop('id', axis=1, inplace=False)
    train_data, rest = train_test_split(data, test_size=train_factor, random_state=42)
    validate_data, test_data = train_test_split(rest, test_size=validate_factor, random_state=42)
    print("Podział zbioru: uczący = ", round(1-train_factor, 2),', walidacyjny = ', round(train_factor*(1-validate_factor),2), ', testowy = ', round(train_factor*validate_factor,2))

    Classifier = NaiveBayes(train_data,"cardio")
    validation_score = Classifier.validate(validate_data)
    test_score = Classifier.validate(test_data)
    print("Dokładność dla zbioru walidacyjnego:", validation_score)
    print("Dokładność dla zbioru testowego:", test_score)
    print(" ")

In [22]:
def cross_validate(data, k):
    data = data.drop('id', axis=1, inplace=False)
    splitted = np.array_split(data, k)
    scores = []
    for i in range(k):
        validation_data = splitted[i]
        rest = splitted.copy()
        rest.pop(i)
        train_data = pd.concat(rest)

        Classifier = NaiveBayes(train_data,"cardio")
        score = Classifier.validate(validation_data)
        scores.append(score)

    print(k, "-krotna walidacja krzyżwowa:")
    print("Dokładność dla zbioru walidacyjnego:", np.mean(scores))
    print(" ")

In [25]:
standard_validation(my_data, 0.66, 0.5)
standard_validation(my_data, 0.9, 0.5)
standard_validation(my_data, 0.7, 0.5)
standard_validation(my_data, 0.5, 0.5)
standard_validation(my_data, 0.3, 0.5)
standard_validation(my_data, 0.1, 0.5)
standard_validation(my_data, 0.999, 0.5)
print("###")
standard_validation(my_data, 0.66, 0.1)
standard_validation(my_data, 0.9, 0.3)
standard_validation(my_data, 0.7, 0.5)
standard_validation(my_data, 0.5, 0.7)
standard_validation(my_data, 0.3, 0.9)

Podział zbioru: uczący =  0.34 , walidacyjny =  0.33 , testowy =  0.33
Dokładność dla zbioru walidacyjnego: 0.5612401066946522
Dokładność dla zbioru testowego: 0.5619588981198076
 
Podział zbioru: uczący =  0.1 , walidacyjny =  0.45 , testowy =  0.45
Dokładność dla zbioru walidacyjnego: 0.5645983645983645
Dokładność dla zbioru testowego: 0.5604117232091324
 
Podział zbioru: uczący =  0.3 , walidacyjny =  0.35 , testowy =  0.35
Dokładność dla zbioru walidacyjnego: 0.5672232529375386
Dokładność dla zbioru testowego: 0.5683542216358839
 
Podział zbioru: uczący =  0.5 , walidacyjny =  0.25 , testowy =  0.25
Dokładność dla zbioru walidacyjnego: 0.5682539682539682
Dokładność dla zbioru testowego: 0.5651044672746162
 
Podział zbioru: uczący =  0.7 , walidacyjny =  0.15 , testowy =  0.15
Dokładność dla zbioru walidacyjnego: 0.5754689754689755
Dokładność dla zbioru testowego: 0.5692574066948827
 
Podział zbioru: uczący =  0.9 , walidacyjny =  0.05 , testowy =  0.05
Dokładność dla zbioru walidac

In [24]:
cross_validate(my_data, 3)
cross_validate(my_data, 4)
cross_validate(my_data, 5)
cross_validate(my_data, 6)
cross_validate(my_data, 7)
cross_validate(my_data, 8)

3 -krotna walidacja krzyżwowa:
Dokładność dla zbioru walidacyjnego: 0.5748257370091923
 
4 -krotna walidacja krzyżwowa:
Dokładność dla zbioru walidacyjnego: 0.5752009749712623
 
5 -krotna walidacja krzyżwowa:
Dokładność dla zbioru walidacyjnego: 0.5763697816904656
 
6 -krotna walidacja krzyżwowa:
Dokładność dla zbioru walidacyjnego: 0.5772355860071209
 
7 -krotna walidacja krzyżwowa:
Dokładność dla zbioru walidacyjnego: 0.5762687839043762
 
8 -krotna walidacja krzyżwowa:
Dokładność dla zbioru walidacyjnego: 0.5756480678169802
 


### Wnioski
Algorytm naiwnej klasyfikacji Bayesa jest to klasyfikator probabilistyczny stosunkowo prosty i szybki w implementacji. Jego główną wadą a jednocześnie podstawą działania jest to że działa w oparciu o założenie niezależności poszczególnych cech/predykatów.

W wyniku tego klasyfikatora, niezależnie od podziału zbioru danych na zbiory uczący, walidacyjny, testowy otrzymywałem bardzo zbliżone rezultaty działania - ok 0.56-0.57 dokładności. Jedynie w przypadku bardzo małej ilości danych uczących - 0.001% wszystkich danych (ok 60 próbek) obserwujemy dość znaczący spadek w dokładności - ok. 52%.

Wprowadzenie k-walidacji krzyżowej nie zmieniło dokładności uzyskiwanej klasyfikacji i wciąż pozostawała ona na sałym poziomie ok. 0,57.

Mimo dość niskiej dokładności klasyfikacji (warto wziąć pod uwagę że nasza klasa jest daną binarną - jest choroba lub jej nie ma, działanie klasyfikatora jest niemalże losowe) naiwny klasyfikator Bayesa działa bardzo szybko nawet z dużą ilością danych, i nie wymaga dużych zasobów.
